In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
# Загружаем данные
raw_data = pd.read_csv('kddcup.data_10_percent.csv')

In [ ]:
# Выбираем интересующие нас столбцы
data = raw_data.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21]].values

In [ ]:
# Закодируем целевую переменную (тип протокола) в бинарный формат
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data[:, 20] = le.fit_transform(data[:, 20])
# TCP - 1, UDP - 2, ICMP - 0, OTHER - 3

In [ ]:
# Разделим данные на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[:, :-1], data[:, -1], test_size=0.2, random_state=0)

In [ ]:
# Создаем модель
encoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu')
])
decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(X_train.shape[1], activation='sigmoid')
])
model = tf.keras.models.Sequential([
    encoder,
    decoder
])

In [ ]:
# Компилируем модель
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Обучаем модель
history = model.fit(X_train.tolist(), y_train.tolist(), epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Проверяем точность модели на тестовой выборке
test_loss, test_acc = model.evaluate(X_test.tolist(), y_test.tolist())
print('Test accuracy:', test_acc)